This is taken from https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

Running the code exactly as on the webpage leads to an error 

`AttributeError: module 'bert' has no attribute 'bert_tokenization'`

on the line:

`FullTokenizer = bert.bert_tokenization.FullTokenizer`

Even though I use conda I had to do the following:
1. pip install bert-for-tf2
2. pip install tf-sentencepiece [ not sure this is necessary ]
3. pip install sentencepiece


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer


from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math


In [ ]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

In [ ]:
model.summary()

In [ ]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
s = "This is a nice sentence."
stokens = tokenizer.tokenize(s)
stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids = get_ids(stokens, tokenizer, max_seq_length)
input_masks = get_masks(stokens, max_seq_length)
input_segments = get_segments(stokens, max_seq_length)

pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])

In [ ]:
FullTokenizer.tokenize("[CLS] This is a sentence [SEP]")

Ok take 2 this is taken from https://github.com/kpe/bert-for-tf2

In [1]:
import bert
import os
model_name = "uncased_L-12_H-768_A-12"
model_dir = "/Users/druss/Downloads/uncased_L-12_H-768_A-12"
model_ckpt = os.path.join(model_dir, "bert_model.ckpt")
bert_params = bert.params_from_pretrained_ckpt(model_dir)
l_bert = bert.BertModelLayer.from_params(bert_params, name="bert")

from tensorflow import keras

max_seq_len = 128
l_input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
l_token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

# using the default token_type/segment id 0
output = l_bert(l_input_ids)                              # output: [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=l_input_ids, outputs=output)
model.build(input_shape=(None, max_seq_len))





In [2]:
model_name = "uncased_L-12_H-768_A-12"
do_lower_case = not (model_name.find("cased") == 0 or model_name.find("multi_cased") == 0)
bert.bert_tokenization.validate_case_matches_checkpoint(do_lower_case, model_ckpt)
vocab_file = os.path.join(model_dir, "vocab.txt")
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
tokens = tokenizer.tokenize("Hello, BERT-World!")
token_ids = tokenizer.convert_tokens_to_ids(tokens)


In [9]:
token_ids

[7592, 1010, 14324, 1011, 2088, 999]

In [11]:
tokens = tokenizer.tokenize("endodontist")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(token_ids)
tokens = tokenizer.tokenize("dentist")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(token_ids)
tokens = tokenizer.tokenize("cardiologist")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(token_ids)

['end', '##odon', '##tist']
[2203, 28716, 16774]
['dentist']
[24385]
['card', '##iol', '##ogist']
[4003, 20282, 22522]


In [8]:
tokens


['end', '##odon', '##tist']